In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = "true"
import tvm
from tvm import relay
from tvm.relay.dataflow_pattern import rewrite
from tvm.relay.dataflow_pattern import *
import numpy as np
from test_model import Model
from tvm.relay import transform


/home/jd/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# pattern = is_op("nn.relu")(is_op("nn.conv2d")(wildcard(), wildcard()))

# # A graph.
# x = relay.var('input')
# w = relay.var('weight')
# conv2d = relay.op.nn.conv2d(x, w)
# relu = relay.op.nn.relu(conv2d)
# print('relu')

In [3]:
# relu

In [4]:
# print(pattern.partition(relu, {'Composite': 'one_layer'}))

In [2]:
model_keras = Model(3, 1, 16)

In [7]:
target = 'cuda'
dev = tvm.cuda()
img_size = 256
input_data = np.random.normal(0,1,(1,img_size,img_size,3)).astype(np.float32)
# tvm result
input_data = input_data.transpose([0, 3, 1, 2])

shape_dict = {"input_1": input_data.shape}
mod, params = relay.frontend.from_keras(model_keras, shape_dict)


In [7]:
mod

#[version = "0.0.5"]
def @main(%input_1: Tensor[(1, 3, 256, 256), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(16), float32], %v_param_4: Tensor[(16), float32], %v_param_5: Tensor[(16), float32], %v_param_6: Tensor[(16), float32], %v_param_7: Tensor[(16, 16, 3, 3), float32], %v_param_8: Tensor[(16), float32], %v_param_9: Tensor[(16), float32], %v_param_10: Tensor[(16), float32], %v_param_11: Tensor[(16), float32], %v_param_12: Tensor[(16), float32]) {
  %0 = nn.conv2d(%input_1, %v_param_1, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]);
  %1 = nn.bias_add(%0, %v_param_2);
  %2 = nn.batch_norm(%1, %v_param_3, %v_param_4, %v_param_5, %v_param_6, epsilon=0.001f);
  %3 = %2.0;
  %4 = nn.leaky_relu(%3, alpha=0.2f);
  %5 = nn.conv2d(%4, %v_param_7, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]);
  %6 = nn.bias_add(%5, %v_param_8);
  %7 = nn.batch_norm(%6, %v_param_9, %v_param_10, %v_param_

In [29]:
from tvm.relay.op.annotation import compiler_begin, compiler_end

def test_extern_ccompiler_single_op():
    @transform.function_pass(opt_level=0)
    class MyAnnotator:
        def transform_function(self, func, mod, dev):
            class Annotator(tvm.relay.ExprMutator):
                def visit_call(self, call):
                    new_args = []
                    for arg in call.args:
                        ann = compiler_begin(self.visit(arg), "ccompiler")
                        new_args.append(ann)
                    new_call = relay.Call(call.op, new_args)
                    return compiler_end(new_call, "ccompiler")

            return Annotator().visit(func)

    x = relay.var("x", shape=(8, 8))
    y = relay.var("y", shape=(8, 8))
    z = x + y
    z = z + relay.const(9.0)
    z = z + relay.const(9.0)
    f = relay.Function([x, y], z)
    x_data = np.random.rand(8, 8).astype("float32")
    y_data = np.random.rand(8, 8).astype("float32")
    mod = tvm.IRModule()
    mod["main"] = f
    mod = MyAnnotator()(mod)
    mod = transform.PartitionGraph()(mod)

    # print(mod)
    # print("---------------")
    # print({"x": x_data, "y": y_data}, (8, 8), x_data + y_data)
    return mod

In [30]:
mm = test_extern_ccompiler_single_op()


In [89]:
type(mm)

tvm.ir.module.IRModule

In [92]:
mod = tvm.IRModule.from_expr(mm['tvmgen_default_ccompiler_main_0'])


In [93]:
mod

#[version = "0.0.5"]
def @tvmgen_default_ccompiler_main_0(%ccompiler_0_i0: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */, %ccompiler_0_i1: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */, Inline=1, Compiler="ccompiler", global_symbol="tvmgen_default_ccompiler_main_0", Primitive=1) -> Tensor[(8, 8), float32] {
  add(%ccompiler_0_i0, %ccompiler_0_i1) /* ty=Tensor[(8, 8), float32] */
}

In [94]:
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)


ValueError: Traceback (most recent call last):
  7: TVMFuncCall
  6: tvm::relay::backend::RelayBuildModule::GetFunction(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, tvm::runtime::ObjectPtr<tvm::runtime::Object> const&)::{lambda(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)#3}::operator()(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*) const
  5: tvm::relay::backend::RelayBuildModule::Build(tvm::IRModule, tvm::runtime::Array<tvm::Target, void> const&, tvm::Target const&, tvm::relay::Executor const&, tvm::relay::Runtime const&, tvm::WorkspaceMemoryPools const&, tvm::ConstantMemoryPools const&, tvm::runtime::String)
  4: tvm::relay::backend::RelayBuildModule::BuildRelay(tvm::IRModule, tvm::runtime::String const&)
  3: tvm::relay::backend::RelayBuildModule::OptimizeImpl(tvm::IRModule)
  2: tvm::relay::backend::BindParamsInModule(tvm::IRModule, std::unordered_map<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> 

In [96]:
z = mm.get_global_vars()[0]
f = relay.Function(z.params, z.body)
mod = tvm.IRModule()
mod['main'] = f
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)

AttributeError: <class 'tvm.ir.expr.GlobalVar'> has no attribute params

In [88]:
for func in mm.functions:
    print(func.astext().split('\n')[-1].split('@')[-1])

tvmgen_default_ccompiler_main_4
tvmgen_default_ccompiler_main_0
tvmgen_default_ccompiler_main_2
main


In [46]:
x = relay.var("x", shape=(8, 8))
y = relay.var("y", shape=(8, 8))
z = mm['tvmgen_default_ccompiler_main_0'](x,y)
f = relay.Function([x, y], z)
mod1 = tvm.IRModule()
mod1['main'] = f

In [69]:
type(z)

tvm.relay.function.Function

In [72]:
z = mm['tvmgen_default_ccompiler_main_0']
z.attrs.global_symbol = 'main'
z = z.with_attr("global_symbol", "main")
mod = tvm.IRModule(z)
# mod['main'] = z
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)



TVMError: Traceback (most recent call last):
  2: TVMFuncCall
  1: tvm::runtime::PackedFuncObj::Extractor<tvm::runtime::PackedFuncSubObj<tvm::runtime::TypedPackedFunc<tvm::IRModule (tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void>, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void>)>::AssignTypedLambda<tvm::{lambda(tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void>, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void>)#2}>(tvm::{lambda(tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void>, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void>)#2}, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)::{lambda(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)#1}> >::Call(tvm::runtime::PackedFuncObj const*, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, tvm::runtime::TVMRetValue)
  0: tvm::runtime::TVMMovableArgValueWithContext_::operator tvm::runtime::Ma

In [ ]:
# func0 = func0.with_attr("global_symbol", "main")


In [49]:
mod1

#[version = "0.0.5"]
def @main(%x: Tensor[(8, 8), float32], %y: Tensor[(8, 8), float32]) {
  %0 = fn (%ccompiler_0_i0: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */, %ccompiler_0_i1: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */, Inline=1, Compiler="ccompiler", global_symbol="tvmgen_default_ccompiler_main_0", Primitive=1) -> Tensor[(8, 8), float32] {
    add(%ccompiler_0_i0, %ccompiler_0_i1) /* ty=Tensor[(8, 8), float32] */
  } /* ty=fn (Tensor[(8, 8), float32], Tensor[(8, 8), float32]) -> Tensor[(8, 8), float32] */;
  %0(%x, %y)
}

In [73]:
z = mm['tvmgen_default_ccompiler_main_0']
f = relay.Function(z.params, z.body)
mod = tvm.IRModule()
mod['main'] = f
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)

AttributeError: 'GraphExecutorFactoryModule' object has no attribute 'body'

In [61]:
f

fn (%ccompiler_0_i0: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */, %ccompiler_0_i1: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */) {
  add(%ccompiler_0_i0, %ccompiler_0_i1) /* ty=Tensor[(8, 8), float32] */
}

In [59]:
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod2, target, params=params)
    # lib = relay.build(mod2, target, params=params,mod_name='tvmgen_default_ccompiler_main_0')


In [40]:
print(mm['tvmgen_default_ccompiler_main_0'])

fn (%ccompiler_0_i0: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */, %ccompiler_0_i1: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */, Inline=1, Compiler="ccompiler", global_symbol="tvmgen_default_ccompiler_main_0", Primitive=1) -> Tensor[(8, 8), float32] {
  add(%ccompiler_0_i0, %ccompiler_0_i1) /* ty=Tensor[(8, 8), float32] */
} /* ty=fn (Tensor[(8, 8), float32], Tensor[(8, 8), float32]) -> Tensor[(8, 8), float32] */


In [39]:
print(f)

fn (%x: Tensor[(8, 8), float32], %y: Tensor[(8, 8), float32]) {
  %0 = fn (%ccompiler_0_i0: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */, %ccompiler_0_i1: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */, Inline=1, Compiler="ccompiler", global_symbol="tvmgen_default_ccompiler_main_0", Primitive=1) -> Tensor[(8, 8), float32] {
    add(%ccompiler_0_i0, %ccompiler_0_i1) /* ty=Tensor[(8, 8), float32] */
  } /* ty=fn (Tensor[(8, 8), float32], Tensor[(8, 8), float32]) -> Tensor[(8, 8), float32] */;
  %0(%x, %y)
}


In [15]:
mod = tvm.IRModule()
mod["main"] = f
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)
    # lib = relay.build(mod2, target, params=params,mod_name='tvmgen_default_ccompiler_main_0')


In [16]:
lib

In [78]:
print(type(legion))


<class 'tvm.relay.function.Function'>


In [80]:
print(type(mm['main']))


<class 'tvm.relay.function.Function'>


In [43]:
# f = type(mm['tvmgen_default_ccompiler_main_0'])
# mod = tvm.IRModule()
# mod["main"] = mm['tvmgen_default_ccompiler_main_0']
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)


In [33]:
# legion = mm['tvmgen_default_ccompiler_main_0']
# mod2 = tvm.IRModule()
# mod2["main"] = legion
with tvm.transform.PassContext(opt_level=3):
    # lib = relay.build(mm['tvmgen_default_ccompiler_main_0'], target, params=params,mod_name='tvmgen_default_ccompiler_main_0')
    lib = relay.build(mm, target, params=params,mod_name='tvmgen_default_ccompiler_main_0')


In [11]:
# legion = mm['tvmgen_default_ccompiler_main_0']
# mod2 = tvm.IRModule()
# mod2["main"] = legion
with tvm.transform.PassContext(opt_level=3):
    # lib = relay.build(mm['tvmgen_default_ccompiler_main_0'], target, params=params,mod_name='tvmgen_default_ccompiler_main_0')
    lib = relay.build(mm, target, params=params,mod_name='tvmgen_default_ccompiler_main_0')


ValueError: Traceback (most recent call last):
  8: TVMFuncCall
  7: tvm::relay::backend::RelayBuildModule::GetFunction(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, tvm::runtime::ObjectPtr<tvm::runtime::Object> const&)::{lambda(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)#3}::operator()(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*) const
  6: tvm::relay::backend::RelayBuildModule::Build(tvm::IRModule, tvm::runtime::Array<tvm::Target, void> const&, tvm::Target const&, tvm::relay::Executor const&, tvm::relay::Runtime const&, tvm::WorkspaceMemoryPools const&, tvm::ConstantMemoryPools const&, tvm::runtime::String)
  5: tvm::relay::backend::RelayBuildModule::BuildRelay(tvm::IRModule, tvm::runtime::String const&)
  4: tvm::runtime::PackedFuncObj::Extractor<tvm::runtime::PackedFuncSubObj<tvm::relay::backend::GraphExecutorCodegenModule::GetFunction(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, tvm::

In [ ]:
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mm, target, params=params)

In [65]:
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mm, target, params=params)


/home/jd/workspace/tvm-v0.9.0/python/tvm/driver/build_module.py:267: UserWarning: target_host parameter is going to be deprecated. Please pass in tvm.target.Target(target, host=target_host) instead.
  warnings.warn(


In [26]:
lib

In [45]:
lib['get_graph_json']

In [62]:
graph_json_raw = lib['get_graph_json']()


In [19]:
import pygraphviz as pgv
import json
def show_graph(json_data, file_name=None):
    if type(json_data) == str:
        json_data = json.loads(json_data)
    A = pgv.AGraph(directed=True)
    for node_idx, node in enumerate(json_data['nodes']):
        for src in node['inputs']:
            # if args.show_size == 1:
            if 1 == 1:
                src_size = 1
                for i in json_data['attrs']['shape'][1][src[0]]:
                    src_size = src_size * i
                
                dst_size = 1
                for i in json_data['attrs']['shape'][1][node_idx]:
                    dst_size = dst_size * i
                # print(src[0], json_data['nodes'][src[0]]['name'], src_size)

                A.add_edge(json_data['nodes'][src[0]]['name'] + '[{}]'.format(src[0]) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]) + "[{}]".format(src_size), node['name'] + '[{}]'.format(node_idx) + '{}'.format(json_data['attrs']['dltype'][1][node_idx]) + "[{}]".format(dst_size))
            else:
                A.add_edge(json_data['nodes'][src[0]]['name'] + '[{}]'.format(src[0]) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]), node['name'] + '[{}]'.format(node_idx) + '{}'.format(json_data['attrs']['dltype'][1][node_idx]))
    if file_name:
        A.draw(file_name + '.png', format='png', prog='dot')

In [63]:
show_graph(graph_json_raw, 'test')
